In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)


In [15]:
df = pd.read_csv("../data/processed/diabetes_preprocessed.csv")
print(df.shape)
df.head()


(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,125.000,33.6,0.627,50.0,1
1,1.0,85.0,66.0,29.0,125.000,26.6,0.351,31.0,0
2,8.0,183.0,64.0,29.0,125.000,23.3,0.672,32.0,1
3,1.0,89.0,66.0,23.0,112.875,28.1,0.167,21.0,0
4,0.0,137.0,40.0,35.0,135.875,43.1,1.200,33.0,1


In [ ]:
# Preprocessing aşamasında temizlenmiş ve kaydedilmiş veri seti yüklendi.
# Bu veri missing value ve outlier işlemleri tamamlanmış halidir.


In [16]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X.shape, y.shape


((768, 8), (768,))

In [ ]:
# Outcome sütunu hedef değişken (y) olarak ayrıldı,
# diğer tüm sütunlar feature seti (X) olarak belirlendi.


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()


((614, 8),
 (154, 8),
 np.float64(0.3485342019543974),
 np.float64(0.35064935064935066))

In [ ]:
# Veri %80 train ve %20 test olacak şekilde bölündü.
# Stratify kullanılarak sınıf oranlarının korunması sağlandı.


In [18]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape


((614, 8), (154, 8))

In [ ]:
# Logistic Regression ve SVM için feature'lar StandardScaler ile ölçeklendi.
# Scaler sadece train verisinde fit edilip test verisine uygulandı.


In [6]:
def evaluate_model(model, X_test_input, y_test_input, proba_available=True):
    """
    proba_available=True ise ROC-AUC için olasılık hesaplar.
    """
    y_pred = model.predict(X_test_input)

    results = {
        "Accuracy": accuracy_score(y_test_input, y_pred),
        "Precision": precision_score(y_test_input, y_pred, zero_division=0),
        "Recall": recall_score(y_test_input, y_pred, zero_division=0),
        "F1": f1_score(y_test_input, y_pred, zero_division=0),
    }

    if proba_available:
        y_prob = model.predict_proba(X_test_input)[:, 1]
        results["ROC_AUC"] = roc_auc_score(y_test_input, y_prob)
    else:
        results["ROC_AUC"] = np.nan

    print("Confusion Matrix:")
    print(confusion_matrix(y_test_input, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test_input, y_pred, zero_division=0))

    return results


In [ ]:
# Tüm modelleri aynı metriklerle değerlendirmek için
# accuracy, precision, recall, F1 ve ROC-AUC hesaplayan fonksiyon tanımlandı.


In [7]:
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

results_lr = evaluate_model(log_reg, X_test_scaled, y_test, proba_available=True)
results_lr


Confusion Matrix:
[[82 18]
 [27 27]]

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.82      0.78       100
           1       0.60      0.50      0.55        54

    accuracy                           0.71       154
   macro avg       0.68      0.66      0.67       154
weighted avg       0.70      0.71      0.70       154



{'Accuracy': 0.7077922077922078,
 'Precision': 0.6,
 'Recall': 0.5,
 'F1': 0.5454545454545454,
 'ROC_AUC': 0.8074074074074075}

In [ ]:
# Logistic Regression modeli baseline olarak eğitildi.
# Model performansı test seti üzerinde değerlendirildi.


In [8]:
svm = SVC(kernel="rbf", probability=True, random_state=42)
svm.fit(X_train_scaled, y_train)

results_svm = evaluate_model(svm, X_test_scaled, y_test, proba_available=True)
results_svm


Confusion Matrix:
[[86 14]
 [24 30]]

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       100
           1       0.68      0.56      0.61        54

    accuracy                           0.75       154
   macro avg       0.73      0.71      0.72       154
weighted avg       0.75      0.75      0.75       154



{'Accuracy': 0.7532467532467533,
 'Precision': 0.6818181818181818,
 'Recall': 0.5555555555555556,
 'F1': 0.6122448979591837,
 'ROC_AUC': 0.8101851851851851}

In [ ]:
# Support Vector Machine modeli eğitildi.
# Non-linear karar sınırı ile Logistic Regression ile karşılaştırıldı.


In [9]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

results_rf = evaluate_model(rf, X_test, y_test, proba_available=True)
results_rf


Confusion Matrix:
[[84 16]
 [21 33]]

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.84      0.82       100
           1       0.67      0.61      0.64        54

    accuracy                           0.76       154
   macro avg       0.74      0.73      0.73       154
weighted avg       0.76      0.76      0.76       154



{'Accuracy': 0.7597402597402597,
 'Precision': 0.673469387755102,
 'Recall': 0.6111111111111112,
 'F1': 0.6407766990291263,
 'ROC_AUC': 0.8203703703703704}

In [ ]:
# Random Forest modeli eğitildi.
# Ensemble yapısı sayesinde non-linear ilişkiler yakalanmaya çalışıldı.


In [10]:
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train, y_train)

results_gb = evaluate_model(gb, X_test, y_test, proba_available=True)
results_gb


Confusion Matrix:
[[85 15]
 [22 32]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.85      0.82       100
           1       0.68      0.59      0.63        54

    accuracy                           0.76       154
   macro avg       0.74      0.72      0.73       154
weighted avg       0.75      0.76      0.76       154



{'Accuracy': 0.7597402597402597,
 'Precision': 0.6808510638297872,
 'Recall': 0.5925925925925926,
 'F1': 0.6336633663366337,
 'ROC_AUC': 0.8214814814814815}

In [ ]:
# Gradient Boosting modeli eğitildi.
# Boosting algoritması kullanılarak model performansı artırılmaya çalışıldı.


In [11]:
results_df = pd.DataFrame(
    [results_lr, results_svm, results_rf, results_gb],
    index=["Logistic Regression", "SVM", "Random Forest", "Gradient Boosting"]
)

results_df


,Accuracy,Precision,Recall,F1,ROC_AUC
Logistic Regression,0.707792,0.600000,0.500000,0.545455,0.807407
SVM,0.753247,0.681818,0.555556,0.612245,0.810185
Random Forest,0.759740,0.673469,0.611111,0.640777,0.820370
Gradient Boosting,0.759740,0.680851,0.592593,0.633663,0.821481


In [ ]:
# Tüm modellerin performans metrikleri tek bir tabloda birleştirildi
# ve karşılaştırmalı olarak incelendi.


In [13]:
results_df.to_csv("../reports/baseline_results.csv", index=True)
print("Saved to ../reports/baseline_results.csv")


Saved to ../reports/baseline_results.csv


In [ ]:
Baseline modeller arasında en iyi genel performansı Gradient Boosting göstermiştir.
Özellikle ROC–AUC değerinin en yüksek olması, modelin diyabetli ve diyabetsiz bireyleri ayırt etme konusunda daha başarılı olduğunu göstermektedir.
Random Forest modeli ise F1-score açısından en iyi sonucu vererek güçlü bir alternatif olmuştur.